# Geoinformatics Project
---
# LEO satellites orbits simulator
###### Angelica Iseni, Emma Lodetti

The **goal** of this project is to compute the position of each LEO (Low Earth orbit) satellite in the constellation for every second of the day.

In this script you will be asked to enter: 
- The **number of orbital planes** of the costellation
- The **number of satellites** per orbital plane
- The **inclination** of the orbital planes with respect to the reference equatorial plane

This information will be used to create almanacs of the constellation satellites.

In [1]:
# Import libraries
import ipywidgets as widgets
from IPython.display import display
from IPython.display import FileLink
import os

# Useful for defining quantities
from astropy import units as u

# Earth focused modules
from poliastro.earth import EarthSatellite
from poliastro.earth.plotting import GroundtrackPlotter
from poliastro.examples import iss
from poliastro.util import time_range

import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import math as mt

### Function definition

In [8]:
# input parameters function
def user_input(var):
    
    n_orbits = num_orbits.value
    n_satellites = num_satellites.value
    inclination = orbit_inclination.value
    
    if n_orbits < 0 or n_satellites < 0 or inclination < 0:
        print('Error: The numbers cannot be negative. Please insert valid numbers.')
    else:
        print(f'You have selected {n_orbits} orbital planes with {n_satellites} satellites each, with an inclination of {inclination} degrees.')

# txt creation function
def create_satellite_txt(i_orbit, i_satellite):
    
    # Satellite name creation as "LEO XXYY", where XX = orbit number and YY = satellite number
    satellite_name = f'LEO{i_orbit:02}{i_satellite:02}'
    
    # M0 computation
    M0 = 360 / num_satellites * (i_satellite - 1) + 360 / num_satellites * ((i_orbit - 1) / num_orbits)
    
    # Omega0 computation
    Omega0 = (i_orbit - 1) * 180 / num_orbits
    
    # Almanac content
    output_content = f'SatelliteName {satellite_name}\nOrbitRadius {orbit_radius}\nOrbitInclination {inclination}\nM0 {M0}\nOmega0 {Omega0}'
    
    # txt file path
    file_path = f'Almanacs{num_orbits:02}{num_satellites:02}{inclination:02}/{satellite_name}.txt'
    
    # Write content in txt file
    with open(file_path, 'w') as file:
        file.write(output_content)


### Input parameters
Insert number of orbits, number of satellites per orbit and inclination of the orbit.

In [9]:
# Widget creation
style = {'description_width': 'initial'}
num_orbits = widgets.IntText(description='Number of orbits:', value=0, style=style)
num_satellites = widgets.IntText(description='Number of satellites per orbit:', value=0, style=style)
orbit_inclination = widgets.FloatSlider(description='Orbit inclination:', min=0, max=90, step=1, style=style)
    
# Submit button
submit_button = widgets.Button(description='Submit')
submit_button.on_click(user_input)

display(num_orbits, num_satellites, orbit_inclination, submit_button)

IntText(value=0, description='Number of orbits:', style=DescriptionStyle(description_width='initial'))

IntText(value=0, description='Number of satellites per orbit:', style=DescriptionStyle(description_width='init…

FloatSlider(value=0.0, description='Orbit inclination:', max=90.0, step=1.0, style=SliderStyle(description_wid…

Button(description='Submit', style=ButtonStyle())

You have selected 5 orbital planes with 5 satellites each, with an inclination of 51.0 degrees.


### Folders and Almanacs creation as txt files

In [10]:
# Convert values to integers
num_orbits = int(num_orbits.value)
num_satellites = int(num_satellites.value)
inclination = int(orbit_inclination.value)

# Almanac folder creation
# "AlmanacsXXYYZZ": XX = number of orbital planes, YY = number of satellites per orbital plane, ZZ = inclination of orbital planes
if not os.path.exists(f'Almanacs{num_orbits:02}{num_satellites:02}{inclination:02}'):
    os.makedirs(f'Almanacs{num_orbits:02}{num_satellites:02}{inclination:02}')

# Satellite positon output folder creation
# "SatellitePositionsXXYYZZ": XX = number of orbital planes, YY = number of satellites per orbital plane, ZZ = inclination of orbital planes
if not os.path.exists(f'SatellitePositions{num_orbits:02}{num_satellites:02}{inclination:02}'):
    os.makedirs(f'SatellitePositions{num_orbits:02}{num_satellites:02}{inclination:02}')

# Orbital radius definition
orbit_radius = 8500

# for cycle on orbit number
for i_orb in range(1, num_orbits + 1):
    # for cycle on satellite number per orbit
    for i_sat in range(1, num_satellites + 1):
        create_satellite_txt(i_orb, i_sat)
        
print(f'Almanacs created successfully!')

Almanacs created successfully!


### Plot Ground Tracks

In [ ]:
# Import and plot coordinates from Output folder

#for file in ['PositionLEO0101.txt', 'PositionLEO0102.txt', 'PositionLEO0103.txt', ...]:
# Load the data from the file
data = pd.read_csv(f'SatellitePositions040444/PositionLEO0103.txt', sep=' ', header=None, names=['latitude', 'longitude', 'height'])

# Create a Basemap instance
m = Basemap(projection='mill', llcrnrlat=-90, urcrnrlat=90, llcrnrlon=-180, urcrnrlon=180, resolution='c')
m.drawcoastlines()
m.drawcountries()
m.drawmeridians(np.arange(-180, 180, 30))
m.drawparallels(np.arange(-90, 90, 30))

# Plot the ground track
x, y = m(data['longitude'], data['latitude'])
m.plot(x, y, 'r-', linewidth=1)

# Add a title and show the plot
plt.title(f'Ground Track')
plt.show()

In [ ]:
data['longitude']*180/mt.pi

In [ ]:
# Load background image
#background_image_path = r'D:\Satellite Code\earth.jpg'
#background_img = plt.imread(background_image_path)

# Create the plot
plt.figure(figsize=(15.2, 8.2))
#plt.imshow(background_img, extent=[-180, 180, -90, 90])

# Create a Basemap instance
m = Basemap(projection='mill', llcrnrlat=-90, urcrnrlat=90, llcrnrlon=-180, urcrnrlon=180, resolution='c')
m.drawcoastlines()
m.drawcountries()
m.drawmeridians(np.arange(-180, 180, 30))
m.drawparallels(np.arange(-90, 90, 30))

# Plot the ground track
plt.scatter(data['longitude']*180/mt.pi, data['latitude']*180/mt.pi, color='red', marker='o', s=4)
plt.xlabel("Longitude (degrees)")
plt.ylabel("Latitude (degrees)")
#plt.title("BEESAT 9 Ground Track ")

# Show the plot
#plt.legend()
plt.grid(True, color='w', linestyle=":", alpha=0.4)
plt.show()

In [ ]:
import pandas as pd
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt

# Read data into a GeoDataFrame
# Assuming your dataframe is named 'data' with columns 'latitude' and 'longitude'
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.longitude, data.latitude))

# Plot the basemap
ax = gdf.to_crs('EPSG:3857').plot(figsize=(10, 10))
cx.add_basemap(ax, crs=gdf.crs)

# Plot the satellite trajectory
gdf.plot(ax=ax, color='red', markersize=10)

plt.title('Satellite Trajectory Over Time')
plt.show()

### Codici di prova

In [ ]:
# prova per far scrivere al codice i vari satelliti ma non va, riproverò

print(f'Orbits has radius = {orbit_radius} km and inclination = {inclination} degrees.')
# for cycle on orbit number
for i_orb in range(1, num_orbits + 1):
    # for cycle on satellite number per orbit
    for i_sat in range(1, num_satellites + 1):
        create_satellite_txt(i_orb, i_sat)
        print(f'Satellite {satellite_name} on orbital plane {i_orb}: M0 = {M0} degrees.')
    print(f'Orbital plane {i_orb}: Omega0 = {Omega0} degrees.')

In [ ]:
# Build spacecraft instance
iss_spacecraft = EarthSatellite(iss, None)
t_span = time_range(
    iss.epoch - 1.5 * u.h, periods=150, end=iss.epoch + 1.5 * u.h
)

In [ ]:
print(iss)

In [ ]:
num_orbits = int(num_orbits.value)
num_satellites = int(num_satellites.value)
inclination = int(orbit_inclination.value)